# 字典和集合

字典是Python中的重要数据结构，并且Python中字典的实现进行了高度优化（如何优化的还没整明白）。

## 泛映射类型

字典属于泛映射类型数据结构，不同于序列类型，字典总是由key-value（键值对）构成。

collections.abc中定义了Mapping和MutableMapping两个抽象基类，这些基类为dict等数据结构定义了形式接口。这些基类主要是作为形式化的文档并定义里构建一个映射类型需要的最基本接口。使用isinstance就可以判断某一个对象是否是广义上的映射类型。

### 散列表以及可散列数据类型

散列函数的使用能够更快速的访问某一特定关键字对应的数据。在Python中通常元素不可变类型都是可散列类型，例如str、bytes以及数值类型。此外，若元组中包含的元素均是可散列类型，则该元组也是可散列类型。Python中，可散列类型必然会实现\_\_hash__()方法，并且具有\_\_eq__()方法。

In [4]:
tt = (1, 2, (3, 4))
print("\n不含可变类型的元组hash：")
print(hash(tt))
tf = (1, 2, frozenset([3, 4]))
print("\n含frozenset的元组hash：")
print(hash(tf))
tl = (1, 2, [3, 4])
print("\n含可变类型的元组hash：")
print(hash(tl))


不含可变类型的元组hash：
-2725224101759650258

含frozenset的元组hash：
-1914358397578938086

含可变类型的元组hash：


TypeError: unhashable type: 'list'

## 字典推导

字典推导类似于列表推导和生成器表达式，是用于初始化字典的有效手段。

## dict & dict-pro-plus-max-super

dict类型是最常用的映射类型，defaultdict和OrderedDict则是dict的变种，这两个类型支持一些特殊的用法。此外还有如ChainMap（可容纳多个映射对象并在进行键查找时，在所有的映射对象中进行查找）、Counter（具有计数器功能，相当适用于计数任务）、UserDict（开箱即用的自定义映射类型基类）等

总的来说，这些变种均具有dict具有的功能，此外还为了应对不同的应用场合提供了更为丰富的功能。

defaultdict对未知键提供了特殊支持，当传入字典中不存在的键获取数据时，dict会直接报错，而defaultdict则会默认为这个新键创建对应的键值对（在建立字典的时候非常非常非常好用，之前都是首先用if判断字典中是否存在该键，然后根据判断结果创建或者修改键值对）。

OrderedDict则对顺序提供了额外支持，普通的dict并不会记录键值对的顺序，或者说dict中的键值对都是无序的，但是OrderedDict则提供了额外的键值对顺序功能支持，支持类似于先入先出的功能。

### get() & setdefault()

get()能够规避对dict中某一个key赋值时，由于key不存在于dict中导致的keyError。但是利用get()方法处理这种情况需要经过多次键查询操作（get()是一次，然后赋值又是一次，并且还需要创建临时变量用于存放取得的值以及对值的操作）

setdefault()则可以一步完成上述工作

### defaultdict

有些时候，希望某个键不存在于映射中也会返回一个默认值，defaultdict实现了\_\_missing__方法，用于应对这种需求。

defaultdict会按照如下步骤处理映射类型中不存在的键

* 调用定义时指定的方法，创建一个新的对象（例如，若默认创建一个列表类型，则会调用list()方法创建一个新list）
* 将新对象作为值，构建键值对并记录到原字典中
* 返回新创建的键值对的值的引用

defaultdict依赖default_factory方法实现上述操作，值得注意的是，default_factory仅会在\_\_getitem__中被调用，对于一个不存在于字典中的键"new_key"，若直接用get()函数获取其对应的值则会返回None。

\_\_getitem__并不会直接调用default_factory，而是按照如下流程进行调用：

* 执行defaultdict["new_key"]，希望获得"new_key"对应的值
* 调用\_\_getitem__()方法，结果没有在键列表中查询到"new_key"
* 调用\_\_missing__()方法，处理未知键"new_key"
* 调用default_factory()方法，赋予"new_key"默认值

上述流程中最为关键的是\_\_missing__()方法的实现。实际上，对于自定义的映射数据类型，若想处理未知键的查询和创建任务，也仅需要实现\_\_missing__()方法

### dict和UserDict

若想建立自定义的映射数据类型，最为朴素的想法是继承dict类。dict作为内置类型，当将其作为父类时，为了避免一些意想不到的错误，需要大量重写方法。UserDict则不同，UserDict实际上不是dict的子类，而是对dict进行了进一步的封装——UserDict中有一个名为data的属性，这一属性是dict的实例，即UserDict中data属性负责存储数据，而其他方法负责处理数据，相较于直接继承自dict类，这样处理能很大程度上避免由于疏忽导致的各种异常以及无限递归（例如由于待查询的键不存在以及疏忽导致的\_\_missing__()无限递归）

### 不可变映射（只读）

在一些应用场合中，或许会希望映射不会被轻易改变，即映射只读。

Python提供了MappingProxyType方法提供这样一种仅只读的类。具体来说，当传入一个映射对象给mappingProxyType，其会返回一个只读的映射视图，并且这一映射视图是动态的 —— 可以通过改变原映射对象来动态改变映射视图中的内容，而不可以直接修改映射视图

In [5]:
# ------------------- 现在向key="3"的键值对中添加元素3

# ------- 方法一 -------
# 一次判断
# 两次键查询
# 不需要临时变量
test_dict = {"1": [1], "2": [1, 2], "3": [1, 2]}
if "3" not in test_dict:
    test_dict["3"] = []
test_dict["3"].append(3)
print(test_dict)

# ------- 方法二 -------
# 两次键查询
# 需要临时变量
test_dict = {"1": [1], "2": [1, 2], "3": [1, 2]}
temp_list = test_dict.get("3", [])
temp_list.append(3)
test_dict["3"] = temp_list
print(test_dict)

# ------- 方法三 -------
# 一次键查询
# 不需要临时变量
test_dict = {"1": [1], "2": [1, 2], "3": [1, 2]}
test_dict.setdefault("3", []).append(3)
print(test_dict)


# ----------------------- dict以及UserDict ----------------------- 
# 实现一个dict，对于这个dict，可以使用字符形式的数字或者直接利用数值进行值查找
# dict[1]和dict["1"]应当返回相同的结果

class strKeyDict0(dict):

    # 对于继承自dict的类需要实现如下方法
    # 1. 实现__missing__()方法，并且若传入的键是数值型还需要转化为str类型进行二次查找
    # 2. 实现__contains__()方法，因为使用 in 关键字进行判断会调用这一方法，
    # 为了保证正确，需要利用传入的键以及转换为str类型的键进行检查，值得注意的是
    # 这里没有使用 key in dict进行检查，因为strKeyDict0本身就是继承自dict
    # key in dict会导致__contains__()无限递归
    # 3. 实现get()方法，虽然[]操作符并不会直接调用get方法，而是调用__getitem__()，
    # 但是get()是非常常见的方法，为了保证在自定义的dict中get()和__getitem__()一致，
    # 需要对get进行重写，这里实际上是由于dict自带的get()方法并不会主动调用__missing__()方法，
    # 当遇到未知的键时，get()方法会直接返回None

    def __missing__(self, key):
        if isinstance(key, str):
            raise KeyError(key)
        return self[str(key)]
    
    def get(self, key, default=None):
        try:
            return self[key]
        except KeyError:
            return default
    
    def __contains__(self, key):
        return key in self.keys() or str(key) in self.keys()


from collections import UserDict
class strKeyDict(UserDict):
    # 对于继承自UserDict的方法，需要实现如下方法
    # 1. 实现__missing__()方法，和strKeyDict(dict)中的实现一致
    # 2. 实现__contains__()方法，不同于继承自dict的strKeyDict0，
    # 继承自UserDict的strKeyDict中利用self.data存储数据，
    # 使得直接用key in dict不会引发无限递归
    # 3. 实现__setitem__()方法，
    # UserDict中__setitem__()的具体功能实现由dict类型的self.data负责
    # 这里仅需要传入相应的键和值即可
    # 4. get()方法，在strKeyDict中不需要重写get()方法，
    # 理由在于userDict继承自Mapping这一超类
    # Mapping.get()和上述strKeyDict0中get()的实现一致

    def __missing__(self, key):
        if isinstance(key, str):
            raise KeyError(key)
        return self[str(key)]
    
    def __contains__(self, key):
        return str(key) in self.data
    
    def __setitem__(self, key, item):
        self.data[str(key)] = item


{'1': [1], '2': [1, 2], '3': [1, 2, 3]}
{'1': [1], '2': [1, 2], '3': [1, 2, 3]}
{'1': [1], '2': [1, 2], '3': [1, 2, 3]}


## 集合

集合在Python中是一个较新的概念，其描述了**唯一**对象的集合。集合有如下特点：
1. 元素的唯一性：集合中不会有两个一样的对象（这里的唯一性指的是hash的唯一性）
2. 集合中的元素是可散列的，但是集合本身是不可散列的
3. 集合允许各种基于集合的二元运算，例如交集、并集以及差集

### 集合推导

类似于列表推导以及生成器表达式

### 集合的操作

Python中的集合支持数学意义上的集合的各类操作，例如差集、并集、交集、对称差集等；此外还支持各种集合/元素之间的判断：属于、子集、真子集等；此外还支持一些其他通用的操作：添加元素、删除元素、迭代等，值得注意的是集合并不支持切片。

## 字典以及集合的实现基础

Python内置的字典以及集合依赖于散列表，散列表的引入一方面给予了字典以及集合快速检索的能力，另一方面也导致字典和集合是无序的并且并不是所有的Python元素均可以作为dict的键或者set的元素。

主要涉及到如下几个问题：

1. Python中的dict、set以及list效率对比
2. 为什么dict和set是无序的
3. 为什么不是所有的Python对象均可以作为dict的键或者set的元素
4. 为什么在dict以及set对象的生命周期中元素的顺序不是一成不变的
5. 为什么不能在迭代循环dict或者set的同时删增元素
6. 为什么dict和set占用的内存空间相较于相同数据的list要大得多

### 字典以及集合中的散列表

dict以及set利用散列表来提高元素搜索效率。具体来说，在dict的散列表中，每一个键值对会占用一个表元，每一个表元由对键的应用和对值的引用两部分组成。

查找特定元素的流程如下：
1. 计算键的散列值
2. 使用散列值的一部分来定位散列表中的一个表元
3. 判断表元是否为空
    * 若为空，抛出KeyError
    * 若不为空，执行第4步
4. 检查是否是期望的元素
    * 若是期望的元素，返回表元中的值
    * 若不是期望的元素，执行第5步
5. 发生冲突，使用散列值的另一部分来定位散列表中的另一行，并返回第3步

添加新元素以及更新现有元素的操作与上述基本一致。不同的是，对于添加新元素，若表元为空则插入元素，否则增加偏移量直到找到下一个为空的表元。此外，**为了保证散列表的稀疏性，当元素数量增添到一定阈值时，Python会自动将这个散列表复制到更大的空间中以避免冲突**。对于更新元素，则在查找到对应的表元后直接对值进行更新。

上述分析实际上已经回答了关于dict和set的若干问题：

1. 正是由于dict和set使用散列表来进行数据存储，在进行元素查找时不需要对所有元素进行遍历，这使得搜索特定元素的效率大大提高。并且由于稀疏性，查找时间不会随着dict和set中元素数量的增加而线性增长。
2. 由于使用散列表进行存储，dict和set中的元素也没有固定的顺序，在添加新元素时原有的顺序可能会被改变
3. dict和set使用散列表进行数据存储，因此这两个数据类型要求元素时可散列的。即，在Python中某元素必须支持\_\_hash__()函数，并且通过\_\_hash__()函数取得的散列值在生命周期中不发生变化。此外，Python还要求可散列的元素支持\_\_eq__()方法以判断相等性。这些条件使得不是所有的对象均可以作为dict的键或者set的元素。
4. 与2类似，由于添加元素后很可能改变原有的顺序，因此在dict和set中讨论元素的顺序没有意义，除非使用类似于OrderedDict这样的特殊类型。
5. 由于元素的顺序不确定，在循环迭代dict或者set的同时删增元素可能会导致跳过某些元素。若使用.keys()、.values()以及.items()等函数对dict进行循环迭代，Python3中这些方法返回的字典视图具有动态的特性。即，循环迭代过程中对dict的改变会实时反馈到循环条件上，这显然会导致不可预测的错误。
6. dict和set是空间换时间的典型例子。散列表需要保证稀疏性以尽可能避免冲突，因此，相较于list，dict和set需要维护更大的内存空间以保证散列表的稀疏性

## 总结

1. Python内置的dict类型在很多任务中承担了重要功能，并且除了基础的dict外，Python标准库也提供了相当多适用于不同应用场合的特殊映射类型（defaultDict、OrderDict、ChainMap等）
2. 多数映射类型均提供了setdefault和update两个方法，前者能够很方便的处理未知键，后者则使得批量更新键值对成为可能
3. \_\_missing__方法为处理未知键提供了接口，通过自定义这一方法可以明确指定遇到未知键时应当如何处理。
4. collections.abs提供了多个映射类型的抽象基类，这些基类可以用于判断对象类型。此外，MappingProxyType可以用来创建不可变映射对象
5. dict和set的实现依赖散列表。散列表利用空间换取时间，这使得dict和set具有相当高的效率，与此同时，散列表的使用也使得在dict和set中讨论元素顺序变得没有意义并且并不是所有元素都可以作为dict的键或者set的元素。